# Training Using Partial Data
In this notebook, we train multiple ML models to capture human capabilities for checking if an image contains another image or not.

In [61]:
import csv
import numpy as np

V = 'A'
c=0
if V == 'A':
    from config import TRAIN_PATH as TRAIN_PATH
    from config import TEST_PATH as TEST_PATH
    b,s,t = 9,10,61
elif V == 'B':
    from config import TRAINB_PATH as TRAIN_PATH
    from config import TESTB_PATH as TEST_PATH
    b,s,t = 62,10,61
elif V == 'C':
    from config import TRAINC_PATH as TRAIN_PATH
    from config import TESTC_PATH as TEST_PATH
    b,s,t = 62,10,61

def load_from_csv(path,ss=None):
    y = []
    X = []
    with open(path,'r') as f:
        reader = csv.reader(f)
        next(reader, None)
        for row in reader:
            if not ss or row[c] in ss:
                y.append(int(row[b]))
                X.append([int(e) for e in row[s:t]])
            
    y = np.array(y)
    X = np.array(X)
    
    return X,y

X_train, y_train = load_from_csv(TRAIN_PATH)
X_test, y_test = load_from_csv(TEST_PATH)

print('Number of training samples:',X_train.shape[0])
print()

print('Distribution of 0s and 1s :')
print('0\t',1-np.sum(y_train)/y_train.shape[0])
print('1\t',np.sum(y_train)/y_train.shape[0])

Number of training samples: 1200

Distribution of 0s and 1s :
0	 0.18833333333333335
1	 0.8116666666666666


In [52]:
import itertools 
import warnings
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore")

def get_grid_search_and_test(estimator, tuned_parameters):
    clf = GridSearchCV(estimator, tuned_parameters, cv=2, scoring='f1_macro', n_jobs=-1)
    clf.fit(X_train, y_train)
    return clf.best_params_,clf.best_score_, clf

m = 0
best = None
ss = None
for ss in itertools.combinations(['6', '15', '10', '7', '8', '16', '14', '18', '19', '11', '17', '9', '12', '5', '13'], 15):
    
    X_train, y_train = load_from_csv(TRAIN_PATH,ss)
    X_test, y_test = load_from_csv(TEST_PATH,ss)
    
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
                         'C': [1, 10, 100, 1000, 10000, 100000]}]
    best_params_, best_score_, _ = get_grid_search_and_test(SVC(), tuned_parameters)
    if best_score_ > m:
        best = best_params_
        m = best_score_
        S = ss
        print(S,best,m)

# ('6',) {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'} 0.7948920682730924
# ('6', '14') {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'} 0.754342372307163
# ('6', '8', '9') {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'} 0.7195983809645624
# ('6', '7', '8', '18', '13') {'C': 10000, 'gamma': 0.0001, 'kernel': 'rbf'} 0.7340249526919209
# ('6', '7', '16', '18', '9', '13') {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'} 0.7242330516672763
# ...
# ('6', '10', '7', '8', '16', '14', '18', '19', '11', '9', '5', '13') {'C': 10000, 'gamma': 0.01, 'kernel': 'rbf'} 0.6162186928522616
# ('6', '15', '7', '8', '16', '14', '18', '19', '11', '9', '12', '5', '13') {'C': 10000, 'gamma': 0.001, 'kernel': 'rbf'} 0.595706144770231
# ('6', '15', '10', '7', '8', '16', '14', '18', '11', '17', '9', '12', '5', '13') {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'} 0.5632029486721866
# ('6', '15', '10', '7', '8', '16', '14', '18', '19', '11', '17', '9', '12', '5', '13') {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'} 0.5453543241430406

# C
# ('6',) {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'} 0.7469252008032128
# ('6', '18') {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'} 0.7161815324478511
# ('6', '7', '18') {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'} 0.6941929077261103
# ('6', '7', '18', '5') {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'} 0.689273324764061
# ('6', '15', '7', '14', '18') {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'} 0.6641720616786991
# ...
# ('6', '15', '10', '7', '8', '14', '18', '11', '17', '9', '12', '5', '13') {'C': 10000, 'gamma': 0.001, 'kernel': 'rbf'} 0.5723381412289125
# ('6', '15', '10', '7', '8', '16', '14', '18', '19', '11', '9', '12', '5', '13') {'C': 100000, 'gamma': 1e-05, 'kernel': 'rbf'} 0.5536459834253613
# ('6', '15', '10', '7', '8', '16', '14', '18', '19', '11', '17', '9', '12', '5', '13') {'C': 100000, 'gamma': 1e-05, 'kernel': 'rbf'} 0.5351864740542237

# B
# ('19',) {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'} 0.6905141155141155
# ('6', '9') {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'} 0.6638738915232688
# ('6', '7', '9') {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'} 0.6434793177194023
# ('8', '18', '11', '9') {'C': 100000, 'gamma': 0.0001, 'kernel': 'rbf'} 0.6473962219978637
# ...
# ('6', '15', '10', '7', '8', '16', '14', '18', '19', '11', '9', '12', '5', '13') {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'} 0.5528425949065511
# ('6', '15', '10', '7', '8', '16', '14', '18', '19', '11', '17', '9', '12', '5', '13') {'C': 10000, 'gamma': 0.001, 'kernel': 'rbf'} 0.534807890196353

('6', '15', '10', '7', '8', '16', '14', '18', '19', '11', '17', '9', '12', '5', '13') {'C': 100000, 'gamma': 1e-05, 'kernel': 'rbf'} 0.5351864740542237


In [62]:
from sklearn.metrics import classification_report
ss = ['6', '15', '10', '7', '8', '16', '14', '18', '19', '11', '17', '9', '12', '5', '13']
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000, 10000, 100000]}]
_, _, clf = get_grid_search_and_test(SVC(), tuned_parameters)
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.27      0.05      0.09        56
           1       0.82      0.97      0.89       244

   micro avg       0.80      0.80      0.80       300
   macro avg       0.54      0.51      0.49       300
weighted avg       0.72      0.80      0.74       300

